
SQLite -- MySQL -- PostgreSQL -- AWS RDS -- BigQuery -- MongoDB -- Neo4J

## [SQLite](http://www.sqlite.org/docs.html)

In [ ]:
import sqlite3
import os

In [ ]:
# connection
os.remove("escola.db") if os.path.exists("escola.db") else None

con = sqlite3.connect('escola.db')
cur = con.cursor()

In [ ]:
# create table
sql_create = ('create table cursos ' +
  '(id integer primary key, ' +
  'titulo varchar(100), ' +
  'categoria varchar(140),' +
  'valor integer)')
cur.execute(sql_create)

In [ ]:
# insert data
sql_insert = 'insert into cursos values (?, ?, ?, ?)'
recset = [(1000, 'Ciencia de Dados', 'Data Science',1500),
          (1001, 'Big Data Fundamentos', 'Big Data',2500),
          (1002, 'Python Fundamentos', 'Analise de Dados',3200),
          (1003, 'Gestão de Dados com MongoDB', 'Big Data',1200),
          (1004, 'R Fundamentos', 'Análise de Dados',6000)]
for rec in recset:
    cur.execute(sql_insert, rec)
con.commit()

In [ ]:
# read data
cur.execute('select * from cursos')
recset = cur.fetchall()
for rec in recset:
    print ('Curso Id: %d, Título: %s, Categoria: %s, Valor: %d ' % rec)

Curso Id: 1000, Título: Ciencia de Dados, Categoria: Data Science, Valor: 1000 
Curso Id: 1001, Título: Big Data Fundamentos, Categoria: Big Data, Valor: 2500 
Curso Id: 1002, Título: Python Fundamentos, Categoria: Analise de Dados, Valor: 3200 
Curso Id: 1003, Título: Gestão de Dados com MongoDB, Categoria: Big Data, Valor: 1200 


In [ ]:
# update data
sql_update = "update cursos set valor = 1000 where valor = 1500"
cur.execute(sql_update)
con.commit()

In [ ]:
# delete data
sql_delete = "delete from cursos where valor = 6000"
cur.execute(sql_delete)
con.commit()

In [ ]:
# close connection
con.close()

## MySQL

In [ ]:
import mysql.connector

# Create connection with db
mydb = mysql.connector.connect(
  host="localhost",
  user="yourusername",
  password="yourpassword",
  database="mydatabase"     #optional
)

# Create cursor from the db
mycursor = mydb.cursor()

# Execute a query
mycursor.execute("Any SQL Query")

# Commit changes
mydb.commit()

# Get results of a query
myresult = mycursor.fetchall()
for row in myresult:
  print(row)

# Close connection
mydb.close()

## PostgreSQL

In [ ]:
import psycopg2

# Create a connection
conn = psycopg2.connect(database="db_name",
                        host="db_host",
                        user="db_user",
                        password="db_pass",
                        port="db_port")

# Create a cursor from a db
cursor = conn.cursor()

# Execute some query
cursor.execute("SELECT * FROM example_table")

# Commit a query
cursor.commit()

# Return only 1 row
cursor.fetchone()

# Return all rows
cursor.fetchall()

# Return exactly n rows
cursor.fetchmany(size=n)

# Close connection
con.close()

## AWS RDS

In [ ]:
import mysql.connector
import sys
import boto3
import os

In [ ]:
#gets the credentials from .aws/credentials
ENDPOINT="authservice-v2.c9r4zedsnlsd.sa-east-1.rds.amazonaws.com"
PORT="3306"
USR="admin"
REGION="us-east-1"
DBNAME="auth"
os.environ['LIBMYSQL_ENABLE_CLEARTEXT_PLUGIN'] = '1'

In [ ]:
# Create session and rds client
session = boto3.Session(profile_name='default')
client = session.client('rds')

In [ ]:
# Password
token = client.generate_db_auth_token(DBHostname=ENDPOINT, Port=PORT, DBUsername=USR, Region=REGION)

In [ ]:
try:
  # Make a connection
  conn = mysql.connector.connect(host=ENDPOINT, user=USR, passwd=token, port=PORT, database=DBNAME, ssl_ca='[full path]rds-combined-ca-bundle.pem')

  # Create a cursor
  cur = conn.cursor()

  # Execute a SQL query
  cur.execute("""Any SQL Query""")

  # Get Result
  query_results = cur.fetchall()

except Exception as e:
  print("Database connection failed due to {}".format(e))

## Google BigQuery

In [ ]:
from google.cloud import bigquery
import google

credentials, your_project_id = google.auth.default(
  scopes=["https://www.googleapis.com/auth/cloud-platform"] #project scope
)

In [ ]:
def bq_create_dataset(dataset_id):
  bigquery_client = bigquery.Client(credentials=credentials, project=your_project_id,)
  dataset_ref = bigquery_client.dataset()

  try:
    bigquery_client.get_dataset(dataset_ref)
  except:
    dataset = bigquery.Dataset(dataset_ref)
    dataset = bigquery_client.create_dataset(dataset)
    print(f'Dataset {dataset.dataset_id} created.')

In [ ]:
def bq_create_table(dataset_id, table_name):
  bigquery_client = bigquery.Client()
  dataset_ref = bigquery_client.dataset(dataset_id)

  # Prepares a reference to the table
  table_ref = dataset_ref.table(table_name)

  try:
    bigquery_client.get_table(table_ref)
  except:
    schema = [
      bigquery.SchemaField('name', 'STRING', mode='REQUIRED'),
      bigquery.SchemaField('age', 'INTEGER', mode='REQUIRED'),
    ]
    table = bigquery.Table(table_ref, schema=schema)
    table = bigquery_client.create_table(table)
    print(f'Table {table.table_id} created.')

In [ ]:
BQ_TABLE_SCHEMA = [
  bigquery.SchemaField('age', 'INTEGER', mode='REQUIRED', description='Age'),
  bigquery.SchemaField('name', 'STRING', description='Name'),
  bigquery.SchemaField('created_at', 'TIMESTAMP', mode='REQUIRED', description='Date and time when the record was created')
]

In [ ]:
def export_items_to_bigquery(dataset_id, table_name):
  # Instantiates a client
  bigquery_client = bigquery.Client()

  # Prepares a reference to the dataset
  dataset_ref = bigquery_client.dataset(dataset_id)

  # Get table
  table_ref = dataset_ref.table(table_name)
  table = bigquery_client.get_table(table_ref)

  # Insert data
  rows_to_insert = [
      (u'Phred Phlyntstone', 32),
      (u'Wylma Phlyntstone', 29),
  ]
  errors = bigquery_client.insert_rows(table, rows_to_insert)
  assert errors == []

## MongoDB

In [ ]:
from pymongo import MongoClient

In [ ]:
conn = MongoClient('localhost', 27017)    # create connection
bdExistentes = conn.list_database_names() #retorna os bd existentes
db = conn.cadastrodb                      #create/define the database to use: 'cadastrodb'
bdName = db.posts.name                    #return db name

In [ ]:
# Collections Ops

'''db.create_collection("mycollection")'''              #create a collection
colecoesExistente = db.list_collection_names()          #return the existing collections
qtdColecao = db.mycollection.estimated_document_count() #counts the qty of documents in the 'mycollection' collection
coll = db["mycollection"]                               #create a connection (so u don't have to write 'db.collectionName' all the time, just 'coll')

In [ ]:
# Insert Data

# data to be ingested
post1 = {"codigo": "ID-9987725","prod_name": "Geladeira","marcas": ["brastemp", "consul", "elecrolux"],
        "data_cadastro": '20/02/2019'}
post2 = {"codigo": "ID-2209876","prod_name": "Televisor","marcas": ["samsung", "panasonic", "lg"],
        "data_cadastro": '18/02/2019'}

# insert the data and the _id and return the id
'''post_id = collection.insert_one(post1).inserted_id'''
'''post_id = collection.insert_one(post2).inserted_id'''

In [ ]:
# looks for certain data in the database and returns it
find = coll.find_one({"prod_name": "Televisor"})

# returns documents from collection 'posts'
for posts1 in db.collection.find():
    print(posts1)

## Neo4j

In [ ]:
# Import the neo4j dependency
from neo4j import GraphDatabase

In [ ]:
# Drive Instance

url = "neo4j://localhost:7687"
username = ''
password = ''

driver = GraphDatabase.driver(connectionString, auth=(username,password)) #create
driver.verify_connectivity()  #test connection

In [ ]:
# Session
with driver.session(database='db_name') as session: #optional parameter
  #auto-commit transactions
  result = session.run(
    "MATCH (n:Node {properties: $name}) RETURN n", #query
    name=prop_value
  )

  #read transactions
  def read_func(tx, prop_value): #aux func
    return tx.run("""
        MATCH (n1:Node1)-[:REALTIONSHIP]->(n2:Node2)
        WHERE n2.properties = $title
        RETURN n1 AS nodeName
        LIMIT 10
    """, title=prop_value)
  result = session.read_transaction(read_func, prop_value="")

  #write transactions
  def write_func(tx, prop_value): #aux func
    return tx.run(
        "CREATE (n:Node {properties: $name})",
        name=prop_value
    )
  result = session.write_transaction(write_func, prop_value="")

  #manually transactions
  with session.begin_transaction() as tx:
    try:
      tx.run(query, **params)
      tx.commit()
    except:
      tx.rollback()

  session.close()

In [ ]:
# Results

for record in result:                           #loop
  r = record['n']
r = [ record['n'] for record in result ]        #list comprehension
first_rec = result.peek()                       #show preview without consuming it
keys = result.keys()                            #get all keys available
single = result.single()                        #expect only a single record - if there is more, a warning will be generated
value = result.value('value_name', 'Default')   #extract a single value - 'Default' is returned when the value didn't exist
values = result.values('val_name1','val_name2') #extract a multiples value - list type is returned
info = result.consume()                         #will consume and return a Result Summary (server, query, execution info)

In [ ]:
## Only inside the auxiliar func

# Node Info - type 'neo4j.graph.Node'
node = [ record['nodeName'] for record in result ][0]

n_id = node.id               #internal ID - number
n_labels = node.labels       #frozenset of labels - list
n_items = node.items()       #properties and it's values - dict
node["prop"]                 #retrieve a properties
node.get("prop", 'Default')  #retrieve a properties - 'Default' is returned when the prop didn't exist

# Relationship Info - type 'neo4j.graph.Relatioship'
relationship = [ record['relationshipName'] for record in result ][0]

n_id = relationship.id               #internal ID - number
n_labels = relationship.labels       #frozenset of type - list
n_items = relationship.items()       #properties and it's values - dict
relationship["prop"]                 #retrieve a properties
relationship.get("prop", 'Default')  #retrieve a properties - 'Default' is returned when the prop didn't exist
start = acted_in.start_node          #internal ID of the node that start the relationship
end = acted_in.end_node              #internal ID of the node that end the relationship

# Path Info - type 'neo4j.graph.Path'
path = [ record['relationship'] for record in result ][0]

start = path.start_node     #internal ID of the node that start the path
end = path.end_node         #internal ID of the node that start the path
size = len(path)            #number of relationship
rela = path.relationships   #array of relationsihp objects
for rel in iter(path):      #can iterated relationship within a path
    print(rel.type)

In [ ]:
# Handling Error
from neo4j.exceptions import Neo4jError, ConstraintError

# Attempt a query
try:
    tx.run(cypher, params)
except ConstraintError as err:  #constaint violation
    code = err.code               #return this format 'Neo.[Classification].[Category].[Title]'
    message = err.message         #return the message
except Neo4jError as err:       #generic erro
    code = err.code               #return this format 'Neo.[Classification].[Category].[Title]'
    message = err.message         #return the message
